# Laboratorium 2: Problemy przy pracy na prawdziwych danych

## Marcin Ogórkiewicz 120674, grupa 1

### Zadanie 1.
Znajdowanie brakujących wartości i ich obsługa
Wczytaj dane z pliku CSV i przeanalizuj, czy w zestawie danych występują brakujące wartości. Wykorzystaj metodę isnull() oraz sum(), aby określić, które kolumny zawierają najwięcej brakujących danych. Następnie wyświetl procent brakujących wartości dla każdej kolumny. Kolejnym krokiem jest wybór odpowiedniej metody obsługi braków – uzupełnij wartości średnią (dla danych liczbowych), najczęściej występującą wartością (dla danych kategorycznych), a także usuń wszystkie wiersze zawierające więcej niż 50% braków.

In [117]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
csv = pd.read_csv("dane_z_problemami.csv") #sczytywanie danych z pliku .csv
empty_cells_pcent = (csv.isnull().sum()/csv.count())*100 #procent brakujących danych w kolumnie
empty_cells_pcent.index.name = "kolumna" # ustawienie nazwy indeksu
empty_cells_pcent.name = "% na" #ustawienie nazawy wartości   
empty_cells_pcent

kolumna
nazwa             0.000000
cena              1.202405
ilość             1.101101
kategoria         0.000000
data_sprzedaży    0.198413
Name: % na, dtype: float64

In [118]:
csv = csv.dropna(thresh = csv.shape[1] / 2) # Usunięcie wierszy z ponad 50% braków
values = {"cena" : np.mean(csv["cena"]), "ilość" : np.mean(csv["ilość"]), "data_sprzedaży" : csv['data_sprzedaży'].value_counts().idxmax()} #słownik uzupełniający brakujące wartości
csv = csv.fillna(value=values) #uzupełnienie brakujących wartości
empty_cells_pcent = (csv.isnull().sum()/csv.count())*100 #procent brakujących danych w kolumnie
empty_cells_pcent.index.name = "kolumna" #ustawienie nazwy indeksu
empty_cells_pcent.name = "% na" #ustawienie nazwy wartości   
empty_cells_pcent

kolumna
nazwa             0.0
cena              0.0
ilość             0.0
kategoria         0.0
data_sprzedaży    0.0
Name: % na, dtype: float64

### Zadanie 2.
Usuwanie duplikatów
Zidentyfikuj i usuń duplikaty w danych. Najpierw użyj metody duplicated() oraz sum(), aby policzyć liczbę powtarzających się wierszy. Następnie usuń duplikaty za pomocą drop_duplicates(). Po wykonaniu tej operacji sprawdź, czy liczba wierszy w DataFrame zmniejszyła się zgodnie z oczekiwaniami.

In [119]:
csv.duplicated().sum() #ilość duplikatów

10

In [120]:
csv = csv.drop_duplicates() #usunięcie duplikatów
csv.duplicated().sum() #ilość duplikatów

0

### Zadanie 3.
Identyfikacja i obsługa wartości odstających
Przeanalizuj dane pod kątem wartości odstających. Skorzystaj z metody describe(), aby sprawdzić, czy w kolumnach liczbowych występują wartości, które znacznie odbiegają od reszty danych (np. wartości maksymalne). Wybierz jedną z kolumn, w której odchylenia są najbardziej widoczne, a następnie zastosuj z-score lub metodę międzykwartylową (IQR), aby wykryć wartości odstające. Usuń wiersze z wartościami odstającymi z wybranej kolumny.

In [121]:
csv.describe()

kolumna,cena,ilość
count,1000.000000,1000.000000
mean,280.043618,492.250798
std,142.904745,267.708894
min,6.000000,19.000000
25%,160.312500,261.000000
50%,288.850000,476.000000
75%,410.920000,666.000000
max,499.930000,985.000000


Odchylenia są najbardziej widoczne w kolumnie ilość

In [122]:
z_scores = zscore(csv["ilość"]) #obliczanie z-score dla kolumny 'ilość'
threshold = 3 #definiowanie progu dla wartości odstających (z-score > 3 lub < -3)
csv = csv[(z_scores < threshold) & (z_scores > -threshold)] #usunięcie wierszy z wartościami odstającymi w kolumnie 'ilość'
csv.describe()

kolumna,cena,ilość
count,1000.000000,1000.000000
mean,280.043618,492.250798
std,142.904745,267.708894
min,6.000000,19.000000
25%,160.312500,261.000000
50%,288.850000,476.000000
75%,410.920000,666.000000
max,499.930000,985.000000


Nic nie usunięto, ponieważ żadna wartość nie znajduje się 3 odchylenia standardowe od średniej

### Zadanie 4.
Korekcja błędów typograficznych i formatowania
Sprawdź, czy w kolumnach tekstowych występują różne formaty lub błędy typograficzne. Na przykład w kolumnie kategoria mogą występować różne zapisy tego samego słowa, np. „Spożywcze” i „spozywcze”. Skorzystaj z metod takich jak str.lower() lub str.capitalize() oraz funkcji replace() w pandas, aby znormalizować format tekstowy.

In [123]:
csv["kategoria"] = csv["kategoria"].str.lower()  #zmiana wszystkich liter na małe
csv["kategoria"] = csv["kategoria"].str.strip() #usuwanie zbędnych spacji
csv["kategoria"] = csv["kategoria"].str.replace('spozywcze', 'spożywcze')  #zastępowanie błędów typograficznych
csv["kategoria"].unique() #sprawdzenie, czy teraz wszystkie kategorie są poprawnie ponazywane

array(['elektronika', 'odzież', 'spożywcze', 'sportowe', 'domowe'],
      dtype=object)

### Zadanie 5.
Konwersja typów danych
Sprawdź typy danych w każdej kolumnie za pomocą metody dtypes. Zidentyfikuj kolumny, w których typy danych nie są zgodne z oczekiwanymi – na przykład, jeśli kolumna cena została wczytana jako typ tekstowy (object), skonwertuj ją na typ liczbowy (float). Dodatkowo, zmień format daty w kolumnie data_sprzedaży na standardowy format datetime, korzystając z pd.to_datetime().

In [124]:
csv.dtypes

kolumna
nazwa              object
cena              float64
ilość             float64
kategoria          object
data_sprzedaży     object
dtype: object

Ilość powinna być liczbą całkowitą, a data powinna być typem datetime

In [125]:
csv['ilość'] = csv['ilość'].astype('int') #konwersja kolumny 'ilość' z float64 na int
csv['data_sprzedaży'] = pd.to_datetime(csv['data_sprzedaży'], errors='coerce')  #konwersja kolumny "data_sprzedaży" na typ datetime
csv.dtypes

kolumna
nazwa                     object
cena                     float64
ilość                      int32
kategoria                 object
data_sprzedaży    datetime64[ns]
dtype: object

### Zadanie 6.
Uzupełnianie brakujących danych przy użyciu interpolacji
Wybierz kolumnę liczbową z brakującymi wartościami i uzupełnij te braki, stosując interpolację (np. metodę liniową). Przeanalizuj, jak zmieniają się wyniki analizy po zastosowaniu interpolacji, porównując je z poprzednimi wynikami.

In [126]:
csv['ilość'].describe()

count    1000.000000
mean      492.242000
std       267.708889
min        19.000000
25%       261.000000
50%       476.000000
75%       666.000000
max       985.000000
Name: ilość, dtype: float64

In [127]:
csv['ilość'] = csv['ilość'].interpolate(method='linear') #interpolacja przy użyciu metody liniowej
csv['ilość'].describe()

count    1000.000000
mean      492.242000
std       267.708889
min        19.000000
25%       261.000000
50%       476.000000
75%       666.000000
max       985.000000
Name: ilość, dtype: float64

Statystyki się nie zmieniły, ponieważ nie było braków w kolumnie, wszystkie kolumny są wypełnione od zadania 1

### Zadanie 7.
Obsługa danych niezgodnych z formatem lub zakresem
Zidentyfikuj wiersze, w których wartości w kolumnie cena przekraczają określony zakres – na przykład większe niż 500 lub mniejsze niż 5. Zastosuj funkcję clip(), aby ograniczyć wartości do zakresu 5–500. Następnie przeanalizuj, w jaki sposób taka korekta wpływa na wyniki obliczeń statystycznych.

In [128]:
csv['cena'].describe()

count    1000.000000
mean      280.043618
std       142.904745
min         6.000000
25%       160.312500
50%       288.850000
75%       410.920000
max       499.930000
Name: cena, dtype: float64

In [129]:
csv['cena'] = csv['cena'].clip(lower=5, upper=500) #przycięcie cen
csv['cena'].describe()

count    1000.000000
mean      280.043618
std       142.904745
min         6.000000
25%       160.312500
50%       288.850000
75%       410.920000
max       499.930000
Name: cena, dtype: float64

Statystyki się nie zmieniły, gdyż ceny już wcześniej były większe od 5 i mniejsze od 500

### Zadanie 8.
Zapis oczyszczonych danych
Na koniec zapisz oczyszczone i przekształcone dane do nowego pliku CSV. Upewnij się, że nowe dane nie zawierają braków, duplikatów ani nieprawidłowych formatów. Zapisz plik z nowymi nazwami kolumn lub ze zaktualizowanymi formatami danych, co ułatwi dalszą analizę i pracę na tych danych.

In [130]:
csv['cena'] = csv['cena'].round(2) #zaokrąglanie wartości do dwóch miejsc po przecinku w kolumnie cena, ponieważ przy pierwszej próbie niektóre z cen miały 8 cyfr po przecinku
csv.to_csv('oczyszczone_dane.csv', index=False)

### Wnioski

Laboratorium nauczyło mnie pracy na danych surowych. Miałem okazję poćwiczyć ich obróbkę oraz analizę. Niektóre zadania trochę zbiły mnie z tropu, ponieważ nie wprowadzały zmian do danych.